In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

## Read the training and test data sets previously split and stored as separage csv files
## The last column labelled '86' is the 0/1 label indicationg benign/malware for each sample row.

df_train = pd.read_csv("data/train_data.csv")
df_test = pd.read_csv("data/test_data.csv")

last_column = df_train.shape[1] - 1 ## index of the label column
X_train = df_train.iloc[:,:last_column]
y_train = df_train.iloc[:,last_column]

X_test = df_test.iloc[:,:last_column]
y_test = df_test.iloc[:,last_column]

In [2]:
import tensorflow as tf
from tensorflow import keras



In [3]:
X_train.shape

(20532, 74)

In [26]:
def create_model(learning_rate=0.00010):
    model = keras.models.Sequential()
    model.add(keras.Input(shape=(74,)))
    model.add(keras.layers.Dense(50,activation='relu'))
    model.add(keras.layers.Dense(1,activation='sigmoid'))
    model.summary()
    opt = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.BinaryCrossentropy()
    model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])
    return model


In [29]:
create_model(learning_rate=0.00010)
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=9, restore_best_weights=True, verbose=1)
model.fit(x=X_train, y=y_train, verbose=0, validation_split=0.2, epochs=200, callbacks=[callback])
model.evaluate(x=X_test, y=y_test)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 50)                3750      
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 51        
Total params: 3,801
Trainable params: 3,801
Non-trainable params: 0
_________________________________________________________________
Restoring model weights from the end of the best epoch.
Epoch 00010: early stopping
275/275 [==============================] - 0s 825us/step - loss: 0.1129 - accuracy: 0.9641


[0.11293693631887436, 0.964090883731842]

In [30]:
yp = model.predict(X_test)
yp = yp > 0.5
print('acc', accuracy_score(y_test, yp))
print('recall', recall_score(y_test, yp))
print('precision', precision_score(y_test, yp))
print('F1', f1_score(y_test, yp))
pd.DataFrame(confusion_matrix(y_test,yp))

acc 0.9640909090909091
recall 0.9597933513027853
precision 0.9689342403628118
F1 0.9643421349582487


,0,1
0,4211,137
1,179,4273
